In [1]:
from gurobipy import Model, GRB

In [2]:
# Assuming hypothetical costs for demonstration purposes
# c[i-1][j-1] represents the cost c(i,j) for company i to provide service to route j
costs = [
    [10, 15, 20, 25, 30, 35],  # Costs for company A
    [12, 18, 24, 30, 36, 42],  # Costs for company B
    [14, 21, 28, 35, 42, 49],  # Costs for company C
    [16, 24, 32, 40, 48, 56]   # Costs for company D
]

In [3]:
# Initialize the model
m = Model("MoosoneeBusRoutes")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [4]:
# Decision variables
x = m.addVars(4, 6, vtype=GRB.BINARY, name="x")  # x(i,j) variable
y = m.addVars(6, vtype=GRB.BINARY, name="y")    # y(j) variable

In [5]:
# Objective Function: Minimize the total cost
m.setObjective(sum(costs[i][j] * x[i,j] for i in range(4) for j in range(6)), GRB.MINIMIZE)

In [7]:
# Constraints
# 1. Only one company per route if opened
for j in range(6):
    m.addConstr(sum(x[i,j] for i in range(4)) == y[j], name=f"company_per_route_{j+1}")

# 2. Each company to at most two routes
for i in range(4):
    m.addConstr(sum(x[i,j] for j in range(6)) <= 2, name=f"max_routes_for_company_{i+1}")

# 3. At least three routes must be opened
m.addConstr(sum(y[j] for j in range(6)) >= 3, name="min_routes")

# 4. Route 2 and 5 both opened or not
m.addConstr(y[1] == y[4], name="route_2_and_5")

# 5. Either route 3 is opened or route 4 but not both
m.addConstr(y[2] + y[3] == 1, name="route_3_or_4")

# 6. Company B (2) constraint with route 1 and 4
m.addConstr(x[1,0] + x[1,3] <= 1, name="companyB_constraint")

# 7. Company A (1) assigned to route 3 implies it to route 5
m.addConstr(x[0,2] <= x[0,4], name="companyA_constraint")

# 8. Company D (4) must be assigned to at least one route
m.addConstr(sum(x[3,j] for j in range(6)) >= 1, name="companyD_min_route")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 30 columns and 104 nonzeros
Model fingerprint: 0xa595cd3c
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 95.0000000
Presolve removed 10 rows and 6 columns
Presolve time: 0.00s
Presolved: 12 rows, 24 columns, 56 nonzeros
Variable types: 0 continuous, 24 integer (24 binary)
Found heuristic solution: objective 77.0000000

Root relaxation: objective 6.050000e+01, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   60.50000 

In [9]:
# Display the solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost: {m.ObjVal}")
    for i in range(4):
        for j in range(6):
            if x[i,j].X > 0.5:  # If company i is assigned to route j
                print(f"Company {i+1} is assigned to Route {j+1}")
else:
    print("No optimal solution found.")

Optimal total cost: 74.0
Company 1 is assigned to Route 3
Company 1 is assigned to Route 5
Company 4 is assigned to Route 2
